# Importar pacotes

In [1]:
import pandas as pd

# Classe para modelagem de linguagem. Nesse casso, escolhemos o count vectorizer.
# ver https://scikit-learn.org/stable/modules/feature_extraction.html para outras abordagens
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# Classe para aprendizado de máquina.
# Escolhemos o algoritmo naive bayes, por ser uma abordagem simples que funciona bem para textos curtos
# Especificamente, utilizaremos o o multinomial naive bayes 
from sklearn.naive_bayes import MultinomialNB
# O seguinte pacote serve para separar um conjunto de dados em duas partes: 
# uma que será utilizada para treinar o modelo e outra que será utilizada para testar o modelo
from sklearn.model_selection import train_test_split
# Essa função serve para medir o quão bom está seu modelo.
from sklearn.metrics import classification_report
# essa classe ajuda a transformar as categorias em números para que o computador consiga processá-las facilmente.
from sklearn.preprocessing import LabelEncoder
# Função para criar pipeline
from sklearn.pipeline import make_pipeline

# Lendo e filtrando os dados

A primeira etapa do processo é ler os dados para que seja possível construir nosso modelo de linguagem e, posteriormente, nosso modelo de aprendizado de máquina. Para isso, usaremos o pandas, que tem uma série de funções e módulos que ajudam a ler e processar dados - incluindo a função `read_csv()`, que lê arquivos csv e transforma eles em DataFrames (o tipo do pandas que funciona como uma tabela).

In [2]:
df = pd.read_csv('../data/sample_products.csv')

In [3]:
# vendo os 10 primeiras linhas do arquivo
df.head(n=10)

,product_id,seller_id,query,search_page,position,title,concatenated_tags,creation_date,price,weight,express_delivery,minimum_quantity,view_counts,order_counts,category
0,11394449,8324141,espirito santo,2,6,Mandala Espírito Santo,mandala mdf,2015-11-14 19:42:12,171.890000,1200.0,1,4,244,NaN,Decoração
1,15534262,6939286,cartao de visita,2,0,Cartão de Visita,cartao visita panfletos tag adesivos copos lon...,2018-04-04 20:55:07,77.670000,8.0,1,5,124,NaN,Papel e Cia
2,16153119,9835835,expositor de esmaltes,1,38,Organizador expositor p/ 70 esmaltes,expositor,2018-10-13 20:57:07,73.920006,2709.0,1,1,59,NaN,Outros
3,15877252,8071206,medidas lencol para berco americano,1,6,Jogo de Lençol Berço Estampado,t jogo lencol menino lencol berco,2017-02-27 13:26:03,118.770004,0.0,1,1,180,1.0,Bebê
4,15917108,7200773,adesivo box banheiro,3,38,ADESIVO BOX DE BANHEIRO,adesivo box banheiro,2017-05-09 13:18:38,191.810000,507.0,1,6,34,NaN,Decoração
5,4336889,3436479,dia dos pais,1,37,Álbum de figurinhas dia dos pais,albuns figurinhas pai lucas album fotos,2018-07-11 10:41:33,49.970000,208.0,1,1,1093,NaN,Lembrancinhas
6,7544556,7118324,arranjo de flores para mesa,1,9,Arranjo de Flores - Orquidias,mini arranjos,2016-04-22 13:34:16,23.670000,207.0,1,5,276,NaN,Decoração
7,10869150,5203458,lembrancinha maternidade,5,18,Kit Aromarizador + sacola / Lembrancinha Mater...,bb lembrancinhas maternidade baby lembranca ma...,2017-10-05 00:26:02,12.710000,55.0,0,33,1178,109.0,Lembrancinhas
8,13193769,2933585,chaveiro dia dos pais,1,35,chaveiro dia dos pais,dia pais,2018-07-04 12:47:49,11.420000,6.0,1,23,72,NaN,Lembrancinhas
9,13424151,8530613,manta personalizada,1,20,Manta para bebê personalizada de Nuvem com nome,nascimento manta baby cha bebe vestido bebe,2018-04-03 16:10:51,107.100000,9.0,1,1,639,26.0,Bebê


Como o arquivo contém vários campos, teremos que filtrar os dados para ficar mais fácil o consumo posterior. Utilizaremos os campos `title` e `category` para nossa análise. Criaremos um modelo que, a partir do campo `title`, tentará "advinhar" qual o campo `category`.

In [4]:
# Aqui, está falando para o pandas selecionar somente os campos "title" e "category".
df = df[['title', 'category']]
df.head(n=10)

,title,category
0,Mandala Espírito Santo,Decoração
1,Cartão de Visita,Papel e Cia
2,Organizador expositor p/ 70 esmaltes,Outros
3,Jogo de Lençol Berço Estampado,Bebê
4,ADESIVO BOX DE BANHEIRO,Decoração
5,Álbum de figurinhas dia dos pais,Lembrancinhas
6,Arranjo de Flores - Orquidias,Decoração
7,Kit Aromarizador + sacola / Lembrancinha Mater...,Lembrancinhas
8,chaveiro dia dos pais,Lembrancinhas
9,Manta para bebê personalizada de Nuvem com nome,Bebê


In [5]:
# O pandas pode ser utilizado para analisar melhor os dados. O método describe nos dá algumas informações importantes.
print(".describe() dos títulos")
print(df['title'].describe())
print('\n------------\n')
print(".describe() das categorias")
print(df['category'].describe())

.describe() dos títulos
count                     38000
unique                    25113
top       Lembrancinha Batizado
freq                         66
Name: title, dtype: object

------------

.describe() das categorias
count             38000
unique                6
top       Lembrancinhas
freq              17524
Name: category, dtype: object


A partir do método .describe() é possível verificar que os títulos são diversos (25113 dos títulos são "únicos") e as categorias são pouco diversas (existem somente 6).

Agora, precisamos transformar os títulos e as categorias em números, de modo que o computador consiga processá-las facilmente. Faremos isso utilzando a classe `LabelEncoder`.

In [6]:
# Em primeiro lugar, instanciamos a classe.
label_encoder = LabelEncoder()
# Em segundo lugar, criamos as "labels" a partir da coluna "category" e adicionamos uma nova coluna chamada "label"
df['labels'] = label_encoder.fit_transform(df['category'])
df['labels'].head(n=10)

0    2
1    5
2    4
3    0
4    2
5    3
6    2
7    3
8    3
9    0
Name: labels, dtype: int64

# Criando o modelo de linguagem
Agora, precisamos criar um modelo de linguagem. Utilizaramos o CountVectorizor para isso. Em poucas palavras, ele implementa uma abordagem denominada *bag of words*, que consiste em compilar a frequência de palvras em um texto.

In [7]:
# Começamos instanciando a classe.
count_vectorizer = CountVectorizer()

# Posteriormente, precisamos criar o modelo. O método fit cria o modelo e o transform transforma 
# os dados passados em vetores. Colocamos o resultado na coluna "vectors"
count_vectorizer.fit(df['title'])

vectors = count_vectorizer.transform(df['title'])

# Dividindo os dados

Antes de criar o modelo de aprendizado, é necessário divir os dados entre dados de treino, que serão utilizados para treinar o modelo, e de teste, que serão utilizados para testar o modelo. Deste modo, garantimos que as inferências do modelo não fique muito "específicas" aos dados utilizados para treiná-lo (pesquisa o termo overfitting).

Vamos chamar o título de x e a categoria de y. A ideia é que o modelo de aprendizado funcione como uma função: f(x) -> y

In [8]:
x_train, x_test = train_test_split(vectors, test_size=0.3, random_state=41)
y_train, y_test = train_test_split(df['labels'], test_size=0.3, random_state=41)

# Criando o modelo de aprendizado de máquina

A criação do modelo de aprendizado segue uma lógica similar a criação do modelo de linguagem. O que estamos fazendo aqui, agora, é deixando o modelo inferir as relações entre as palavras da coluna "vectors" para as categorias no campo "labels".

In [9]:
# Instanciando a classe
multinomial_nb = MultinomialNB()
# Nesta linha a gente "treina" o modelo.
multinomial_nb.fit(X=x_train, y=y_train)

MultinomialNB()

# Verificando métricas do modelo

Basicamente, você precisa verificar as seguintes métricas: Acurácia, precisão, recall e f1 (https://medium.com/@kohlishivam5522/understanding-a-classification-report-for-your-machine-learning-model-88815e2ce397). Podemos fazer isso com a função `classification_report`. 

Para usar essa função precisamos, antes, classificar nossos dados de teste (o *x_test*).

In [10]:
predicted_y_test = multinomial_nb.predict(x_test)
report = classification_report(y_test, predicted_y_test)
print(report)
print({n: v for n, v in enumerate(label_encoder.classes_)})

              precision    recall  f1-score   support

           0       0.83      0.80      0.82      2024
           1       0.92      0.88      0.90       311
           2       0.87      0.85      0.86      2596
           3       0.84      0.93      0.89      5310
           4       0.86      0.49      0.62       326
           5       0.82      0.57      0.67       833

    accuracy                           0.85     11400
   macro avg       0.86      0.75      0.79     11400
weighted avg       0.85      0.85      0.84     11400

{0: 'Bebê', 1: 'Bijuterias e Jóias', 2: 'Decoração', 3: 'Lembrancinhas', 4: 'Outros', 5: 'Papel e Cia'}


# Criando o pipeline

O pipeline nada mais é do que uma maneira de unir todas as etapas anteriores em um único sistema. A função  `make_pipeline` (https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html) funciona da seguinte maneira: ele toma uma quantidade indeterminada de argumentos (representada no python pelo `*`). Os argumentos passados funcionarão como se eles estivessem em uma lista: a ordem de sua disposição é a ordem em que eles serão executados.

In [11]:
x_train, x_test = train_test_split(df['title'], test_size=0.3, random_state=42)
y_train, y_test = train_test_split(df['labels'], test_size=0.3, random_state=42)

count_vectorizer = CountVectorizer() # modelagem de linguagem
multinomial_nb = MultinomialNB() # modelo de aprendizado de máquina

pipeline = make_pipeline(count_vectorizer, multinomial_nb)

pipeline.fit(X=x_train, y=y_train) # nessa linha o pipeline 1. cria o modelo de linguagem 2. treina o modelo de aprendizado de máquina

predicted_y_test = pipeline.predict(X=x_test) # Predizindo os resultados dos nossos dados de teste

report  = classification_report(y_true=y_test, y_pred=predicted_y_test)

print(report)

              precision    recall  f1-score   support

           0       0.82      0.81      0.82      2091
           1       0.94      0.87      0.90       273
           2       0.86      0.86      0.86      2584
           3       0.84      0.92      0.88      5265
           4       0.85      0.49      0.62       335
           5       0.81      0.56      0.66       852

    accuracy                           0.85     11400
   macro avg       0.86      0.75      0.79     11400
weighted avg       0.84      0.85      0.84     11400



# Como salvar a pipeline e as métricas em arquivos

In [12]:
import os
import pickle
# Aqui temos que ler as variáveis de ambiente (que estão no arquivo .env)
metrics_path = os.getenv('METRICS_PATH').strip('"') # os .strip('"') foram adicionados para remover os caracteres " no começo e final da string
model_path = os.getenv('MODEL_PATH').strip('"')

with open(metrics_path, 'w') as f: 
    # aqui estamos abrindo um arquivo no modo escrita (w), colocando ele na variável f (no bloco identado)
    # e escrevemos o conteúdo do report no arquivo
    f.write(report)
    
with open(model_path, 'wb') as f:
    # aqui estamos abrindo um arquivo no modo escrita binária (wb),
    # colocando ele na variável f (no bloco identado) e escrevemos o conteúdo do report no arquivo
    
    # Antes de escrever a pipeline, precisamos convertê-lo para um formato binário que possa ser lido, posteriormente,
    # pelo python. Fazemos isso usando a biblioteca pickle e a função dump (ao invés de usar f.write).
    pickle.dump(pipeline, f)